# Jersey City Vacant and Abandoned Properties (APRA)

In [ ]:
# !pip install folium pandas geopandas geopy nominatim openpyxl

In [ ]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopandas.tools import geocode
import geopandas as gpd
import json
import datetime

# DATA PREP

In [ ]:
picklefile = f'./cache/gdf_av.pkl'

print (f'started at {datetime.datetime.now()}')
try:
    
    gdf_av = pd.read_pickle(picklefile)
    print(f'Read gdf_av with shape {gdf_av.shape} from {picklefile}')

except:
    
    print ('Picklefile not found, regenerating... this make take a while.')
    
    # load and prepare the dfs 
    df_a = pd.read_excel(f'./data/apra-2014-2021.xlsx')
    df_v = pd.read_excel(f'./data/vacant-2014-2021.xlsx')

    df_a['full_address'] = df_a.street_address + ", Jersey City, NJ"
    df_v['full_address'] = df_v.street_address + ", Jersey City, NJ"

    # geocode them
    geolocator = Nominatim(timeout=10, user_agent = "atownsend@cornell.edu")

    df_a['gcode'] = df_a.full_address.apply(geolocator.geocode)
    df_v['gcode'] = df_v.full_address.apply(geolocator.geocode)

    # print a warning and list of NOT geocoded
    not_geocoded_a = df_a[df_a['gcode'].isna()]
    print ('NOT GEOCODED — ABANDONED LIST')
    display(not_geocoded_a)

    not_geocoded_v = df_v[df_v['gcode'].isna()]
    print ('NOT GEOCODED — VACANT LIST')
    display(not_geocoded_v)

    # combine the two geocoded dataframes
    df_av = pd.concat([df_a,df_v])

    # make a GeoDataFrame
    # drop any that weren't geocoded, to avoid errors below
    df_av = df_av[df_av['gcode'].notna()]
    df_av['lat'] = [g.latitude for g in df.gcode]
    df_av['lon'] = [g.longitude for g in df.gcode]
    gdf_av = gpd.GeoDataFrame(df_av, geometry=gpd.points_from_xy(df_av.lon, df_av.lat))


    # pickle the combined df
    pd.to_pickle(gdf_av, picklefile)
    print(f'Wrote gdf_av with shape {gdf_av.shape} to {picklefile}')

print (f'finished at {datetime.datetime.now()}')

# EXPLORATION

# PRESENTATION

***

# PARKING LOT

### 1 standard markers map

In [ ]:
my_map = folium.Map(
    location=[40.7128,-74.1],
    tiles='Stamen Toner',
    zoom_start=14)

for index, row in df_vacant_2021.iterrows():
    folium.Marker(location=(row['lat'],row['lon']), popup=str(row.number + row.address), icon=folium.Icon(color='orange',icon='home', icon_color='#FFffff')).add_to(my_map) 
    
for index, row in df_abandoned_2021.iterrows():
    folium.Marker(location=(row['lat'],row['lon']), popup=str(row.address), icon=folium.Icon(color='red',icon='home', icon_color='#FFffff')).add_to(my_map)
    
display(my_map)

### 2 heatmap

In [ ]:
# folium heat map
from folium import plugins

heat_map = folium.Map(
    location=[40.7128,-74.1],
    tiles='Stamen Toner',
    zoom_start=14)

heat_data = [[ row.lat, row.lon] for index, row in df_vacant_2021.iterrows() ]

for index, row in df_abandoned_2021.iterrows():
    heat_data.append([ row.lat, row.lon])
                                        


plugins.HeatMap(heat_data).add_to(heat_map)

display(heat_map)


### 3 parcel polygons map

In [ ]:
# convert the dataframes to geodataframes
# Convert the DataFrame's content (e.g. Lat and Lon columns) into appropriate Shapely geometries first and then use them together with the original DataFrame to create a GeoDataFrame.

from geopandas import GeoDataFrame
from shapely.geometry import Point

In [ ]:
# abandoned
geometry_abandoned = [Point(xy) for xy in zip(df_abandoned_2021.lon, df_abandoned_2021.lat)]
# df_abandoned_2021 = df_abandoned_2021.drop(['lon', 'lat'], axis=1)
gdf_abandoned_2021 = GeoDataFrame(df_abandoned_2021, crs="EPSG:4326", geometry=geometry_abandoned)

In [ ]:
# vacants
geometry_vacant = [Point(xy) for xy in zip(df_vacant_2021.lon, df_vacant_2021.lat)]
# df_vacant_2021 = df_vacant_2021.drop(['lon', 'lat'], axis=1)
gdf_vacant_2021 = GeoDataFrame(df_vacant_2021, crs="EPSG:4326", geometry=geometry_vacant)

In [ ]:
# load the parcel map shapefile as a gdf
parcelmap = gpd.read_file(r'../../_gis/maps/NJ/_017_HUDSON/parcels_shp_dbf_Hudson/HudsonCountyParcels.shp')


#### TODO spatial join parcelmap and vacants and abandoned?

In [ ]:
parcelmap.plot(figsize=(17,17))

----------------------------------------------------------------------------------------------------

## TODO a map using marker clusters (what's the mapped variable?)

In [ ]:
# # folium marker clusters
# from folium.plugins import MarkerCluster
# locations = []

# # City location geometries to a list of latlongs pairs  
# for idx, row in gdf.iterrows():
#     locations.append([row['geometry'].y, row['geometry'].x])

#     # Empty canvas
# m = folium.Map(
#     location=[40.7128,-74.1],
#     tiles='Stamen Toner',
#     zoom_start=12)

# # Markercluster
# m.add_child(MarkerCluster(locations=locations))
# m

## TODO a chloropleth on parcels what's the mapped variable?)

In [ ]:
# chloropleth mapping in Folium
# https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0

In [ ]:
# # parcels map
# style_function = lambda x: {'fillColor': '#00ff00'}

# jc_parcels_gdf = geopandas.read_file('https://njgin.state.nj.us/download2/parcels/parcels_shp_dbf_Hudson.zip')
# folium.GeoJson(data=jc_parcels_gdf["geometry"], style_function=style_function).add_to(map)
# map